## Tagebuch

### 2023-06-25

Motivierende Idee:
- Einen Graph aus Personen/Orten bauen, die in Tagesschau-Artikeln gemeinsam erwähnt wurden
- Eventuell nach Entfernung der Erwähnung gewichten
- Lustige Netzwerkanalysen im Graph durchführen
- Visualisieren!

#### Erster Ansatz

spaCy anwerfen und NER betreiben!


## Aufbauen des Datenkorpus

### Step 0: Einrichten der Umgebung

In [2]:
from collections import namedtuple
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
import json
import jsonlines as jl
import networkx as nx
import numpy as np
import pandas as pd
from pathlib import Path
import polars as pl
import shutil
import spacy
from tqdm import tqdm

# Path to the root of the project
_bd = (Path(".") / "..").resolve()

# Data directory
_dd  = _bd / "data"




### Step 1: Sammeln von Nachrichtenartikeln

Artikel von [Tagesschau](https://www.tagesschau.de/) sammeln: `https://www.tagesschau.de/archiv?datum=YYYY-MM-DD` -> z.B.: `https://www.tagesschau.de/archiv?datum=2023-06-22`

Scraping mit [Scrapy](https://scrapy.org/):

```bash
cd newsscrape
scrapy crawl tagesspider -O ../data/1_tagesschau.jsonl
```

```bash
2023-07-04 23:55:35 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 8868909,
 'downloader/request_count': 28108,
 'downloader/request_method_count/GET': 28108,
 'downloader/response_bytes': 2919744022,
 'downloader/response_count': 28108,
 'downloader/response_status_count/200': 25457,
 'downloader/response_status_count/301': 2651,
 'dupefilter/filtered': 2785,
 'elapsed_time_seconds': 3651.299491,
 'feedexport/success_count/FileFeedStorage': 1,
 'finish_reason': 'finished',
 'finish_time': datetime.datetime(2023, 7, 4, 21, 55, 35, 767939),
 'httpcompression/response_bytes': 10862673643,
 'httpcompression/response_count': 25457,
 'item_scraped_count': 25001,
 'log_count/DEBUG': 53115,
 'log_count/ERROR': 33,
 'log_count/INFO': 71,
 'offsite/domains': 2,
 'offsite/filtered': 830,
 'request_depth_max': 1,
 'response_received_count': 25457,
 'scheduler/dequeued': 28108,
 'scheduler/dequeued/memory': 28108,
 'scheduler/enqueued': 28108,
 'scheduler/enqueued/memory': 28108,
 'spider_exceptions/AttributeError': 33,
 'start_time': datetime.datetime(2023, 7, 4, 20, 54, 44, 468448)}
```

Einen Blick auf die Daten werfen:

In [2]:
article_file = f"{_dd}/1_tagesschau_20230704.jsonl"

df_texts = pd.read_json(open(article_file, "r", encoding="utf8"), lines=True)

print(df_texts.shape)

df_texts.sample(5)

(25001, 6)


,tstamp,title,date,shorttext,url,fulltext
22593,1683672539,Was läuft falsch in Serbien?,10.05.2023 • 00:48 Uhr,Nach zwei Amokläufen mit 17 Toten gärt es in S...,/ausland/europa/serbien-gewalt-100.html,Nach zwei Amokläufen mit 17 Toten gärt es in ...
7774,1624512227,"Von Bienen, Bauern und Baumpflege",24.06.2021 • 07:23 Uhr,Monatelang wurde verhandelt - heute stimmt der...,/inland/innenpolitik/insektenschutz-107.html,Monatelang wurde verhandelt - heute stimmt de...
11182,1661231037,Einzelhandel warnt vor dunklen Innenstädten,23.08.2022 • 07:03 Uhr,Am 1. September soll die Energieeinsparverordn...,/wirtschaft/verbraucher/innenstaedte-beleuchtu...,Am 1. September soll die Energieeinsparverord...
23646,1685996941,Wall Street schaltet einen Gang zurück,05.06.2023 • 22:29 Uhr,Zum Wochenstart tendierte die Wall Street unei...,/wirtschaft/finanzen/marktberichte/marktberich...,Zum Wochenstart tendierte die Wall Street une...
3052,1530363905,Kongo will Ölbohrungen erlauben,30.06.2018 • 15:05 Uhr,In den Nationalparks im Kongo leben vom Ausste...,/ausland/kongo-oelbohrungen-101.html,In den Nationalparks im Kongo leben vom Ausst...


Datumsangaben ergänzen:

In [26]:
# df_texts["timestamp"] = pd.to_datetime(df_texts["tstamp"], unit="s", utc=True).dt.tz_convert("Europe/Berlin")
# df_texts.drop(columns=["tstamp", "date"], inplace=True)
df_texts.sample(5)


,title,shorttext,url,fulltext,timestamp
13994,Sánchez löst nach Wahlschlappe Parlament auf,Bei den Regional- und Kommunalwahlen erzielten...,/ausland/europa/spanien-neuwahl-100.html,Bei den Regional- und Kommunalwahlen erzielte...,2023-05-29 13:19:10+02:00
4430,+ Russland zeigt offenbar falsche Atombelege +,Russland hat mit offenbar falschen Fotos versu...,/newsticker/liveblog-ukraine-mittwoch-175.html,Russland hat mit offenbar falschen Fotos vers...,2022-10-26 22:20:17+02:00
1624,Afghanistan verdient weiter Unterstützung,Für Afghanistans Niedergang tragen die USA und...,/kommentar/afghanistan-jahrestag-101.html,Für Afghanistans Niedergang tragen die USA un...,2022-08-15 18:36:03+02:00
2097,Indiana erlässt fast vollständiges,In Indiana sind Abtreibungen künftig nur nach ...,/ausland/amerika/indiana-abtreibungsverbot-101...,In Indiana sind Abtreibungen künftig nur nach...,2022-08-06 09:24:55+02:00
12478,Kirchen müssen viele Gebäude aufgeben,"Kirchen, Pfarrhäuser und weitere Gebäude sind ...",/inland/gesellschaft/kirchen-immobilien-100.html,"Kirchen, Pfarrhäuser und weitere Gebäude sind...",2023-05-02 11:29:44+02:00


In [19]:

df_texts["fulltext"].notna().sum()

15123

## Step 2: Extrahieren von Named Entities

Spacy vorbereiten. Wir verwenden das größte deutsche Modell.

Herausforderung I: Der erste Anlauf mit Spacys Standard-NER ergibt leider kein gutes Resultat. Spacy erkennt zwar Entitäten und ordnet sie auch häufig richtig zu, löst aber keine Ambiguitäten auf. "Obama", "Barack Obama", "Barack Hussein Obama" und "Präsident Obama" werden als vier verschiedene Entitäten erkannt. Das ist für unsere Zwecke nicht hilfreich.

Es ist ein Modell erforderlich, das *Entity Disambiguation* bzw. *Entity Linking* unterstützt.  spaCy hat sowas, aber man braucht: 

- eine Knowledge Base
- eine Funktion, die plausible Kandidaten aus der KB zieht,
- ein ML-Modell, das anhand des lokalen Kontexts den wahrscheinlichsten Kandidaten auswählt

Das ist zuviel Aufwand, wir suchen also nach einer Alternative. Folgende Kandidaten sind in der engeren Auswahl:

- [Lucaterre/spacyfishing: A spaCy wrapper of Entity-Fishing (component) for named entity disambiguation and linking on Wikidata](https://github.com/Lucaterre/spacyfishing) ([kermitt2/entity-fishing: A machine learning tool for fishing entities](https://github.com/kermitt2/entity-fishing))
- [amazon-science/ReFinED: ReFinED is an efficient and accurate entity linking (EL) system.](https://github.com/amazon-science/ReFinED)
- [Improving Named Entity Disambiguation using Entity Relatedness within Wikipedia | by Will Seaton | Towards Data Science](https://towardsdatascience.com/improving-named-entity-disambiguation-using-entity-relatedness-within-wikipedia-92f400ee5994)
- [Introducing the Kensho Derived Wikimedia Dataset | by Gabriel Altay | Kensho Blog](https://blog.kensho.com/announcing-the-kensho-derived-wikimedia-dataset-5d1197d72bcf)
- [Kensho Derived Wikimedia Dataset | Kaggle](https://www.kaggle.com/datasets/kenshoresearch/kensho-derived-wikimedia-data?select=property_aliases.csv)
- [kdwd_wikidata_introduction | Kaggle](https://www.kaggle.com/code/kenshoresearch/kdwd-wikidata-introduction)

Wir nehmen @sw-entity-fishing, das per @sw-spacyfishing in Spacy integriert werden kann.

In [3]:
nlp = spacy.load("de_core_news_lg")
# nlp = spacy.load("en_core_web_sm")
# nlp.add_pipe("entityfishing", config={"language": "de", "api_ef_base": "http://nerd.huma-num.fr/nerd/service"})
nlp.add_pipe("entityfishing", config={"language": "de", "api_ef_base": "http://127.0.0.1:8090/service"})

c:\Users\p\miniconda3\envs\mbb-asud-proto-tagesschau\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Single-Threaded NERD

Test mit 100 Artikeln.

In [29]:
text1 = """
Eleven Palestinians have been killed since hundreds of Israeli troops backed by drones entered the camp on Monday. The agencies say infrastructure is being damaged and medics have been stopped from reaching all the injured. Palestinian militant groups meanwhile said a car-ramming and stabbing attack in Israel was a response to the raid. Israeli authorities said seven people were injured on a busy shopping street in Tel Aviv and that the attacker was a Palestinian man from the West Bank. Israel's Prime Minister, Benjamin Netanyahu, said: "Whoever thinks that such an attack will deter us from continuing our fight against terrorism is mistaken."
"""

print(extract_entities(text1))

{}


In [12]:
def entity_list(doc) -> list[dict]:
    return [
        {
            "ner_text": ent.text,
            "label": ent.label_,
            "wd_id": ent._.kb_qid,
            "wd_url": ent._.url_wikidata,
            "score": ent._.nerd_score,
        }
        for ent in doc.ents
    ]

In [14]:
df_text_1 = df_texts.iloc[0:100, :]
with jl.open(f"{_dd}/2_processed-localhost.jsonl", "w") as f_out:
    for article in tqdm(df_text_1.loc[:, ["tstamp", "url", "fulltext"]].itertuples()):
        doc = nlp(article.fulltext)
        entities = entity_list(doc)
        if entities:
            f_out.write(dict(_index=article[0], tstamp=article.tstamp, url=article.url, entities=entities))

1it [00:00,  6.60it/s]

Pandas(Index=0, tstamp=1656638781, url='/ausland/europa/dw-tuerkei-103.html', fulltext=' In der Türkei ist der Internet-Zugang zum Programm der Deutschen Welle gesperrt worden. Als Grund gab die Aufsichtsbehörde eine fehlende Sendelizenz an. Auch Voice of America ist nicht mehr erreichbar.  Die türkische Aufsichtsbehörde für den Rundfunk (RTÜK) hat den Zugang zur Deutschen Welle in der Türkei gesperrt. "Der Zugang zum türkischsprachigen Dienst der Deutschen Welle (DW), DW Turkce, und Voice of America (VOA), die keine Genehmigungen beantragt haben, wurde auf Antrag des RTÜK-Vorstands vom Strafgerichtshof in Ankara blockiert", teilte Ilhan Tasci, Vorstandsmitglied von RTÜK, auf Twitter mit. Eine Sperrung der Internetangebote war erwartet worden, sie droht der DW, VOA und anderen Auslandssendern seit Monaten. Im Februar hatte RTÜK unter Verweis auf das Medienregulierungsgesetz des Landes bekannt gegeben, dass die Sender Lizenzen für On-Demand-Angebote beantragen müssten. Die DW hatte ange

2it [00:00,  7.10it/s]

Pandas(Index=2, tstamp=1656891126, url='/ausland/europa/ukraine-lyssytchansk-selenskyj-101.html', fulltext=' Trotz des Rückzug der Armee will der ukrainische Präsident die Stadt nicht verloren geben. Russland hatte zuvor die vollständige Einnahme gemeldet. Lyssytschansk war die letzte größere Bastion der Ukrainer in Luhansk.  Der ukrainische Präsident Wolodymyr Selenskyj sieht die Stadt Lyssytschansk im Osten des Landes trotz des Rückzugs der ukrainischen Armee noch nicht als verloren an. "Wenn das Kommando unserer Armee Menschen von bestimmten Punkten der Front abzieht, wo der Feind den größten Feuervorteil hat, insbesondere Lyssytschansk, bedeutet das nur eins: Dass wir dank unserer Taktik, dank der verstärkten Versorgung mit modernen Waffen, zurückkommen werden", sagte er am Sonntag in einer Videobotschaft. Die ukrainische Armee bewege sich vorwärts - sowohl im Gebiet Charkiw im Osten, als auch im Gebiet Cherson im Süden und auf dem Schwarzen Meer. Die jüngst wiedererlangte Schlange

3it [00:00,  4.01it/s]

...

98it [02:49,  1.17s/it]

Pandas(Index=98, tstamp=1657714837, url='/wirtschaft/benzin-e10-diesel-preise-adac-101.html', fulltext=' Der langsame Sinkflug der Spritpreise setzt sich fort. Dennoch kritisiert der ADAC die Preise für Benzin und Diesel als "nach wie vor stark überhöht".  Die Preise an den Tankstellen geben weiter langsam nach. Berechnungen des ADAC zufolge ist Superbenzin der Sorte E10 die fünfte Woche in Folge billiger geworden. Gestern kostete ein Liter im bundesweiten Tagesdurchschnitt 1,826 Euro, das waren 1,7 Cent weniger als eine Woche zuvor. Bei den Dieselpreisen ging es nach den Untersuchungen des ADAC in der dritten Woche in Folge abwärts auf 1,963 Euro am Dienstag - ein Rückgang um 2,6 Cent. Dennoch hält der Verkehrsclub die Preise nach wie vor für "stark überhöht". Treiber des aktuellen Rückgangs dürften unter anderem die Ölpreise sein, die wegen der Konjunktursorgen seit einigen Wochen tendenziell nachgeben. Kostete ein Barrel der Nordsee-Sorte Brent vor einigen Wochen noch mehr als 120 U

99it [02:50,  1.02s/it]

Pandas(Index=99, tstamp=1657715925, url='/ausland/afrika/mali-minusma-festnahmen-101.html', fulltext=' Die Regierung in Mali hat 49 ivorische Soldaten festgenommen. Die Machthaber vermuteten einen Putschversuch - tatsächlich kamen die Männer zum Schutz eines UN-Stützpunktes. Verteidigungsministerin Lambrecht übte Kritik.  Die Militärregierung im westafrikanischen Mali hat die Wachmannschaft eines Camps der UN-Truppe MINUSMA auf dem Flughafen der Hauptstadt Bamako festnehmen lassen. Die Männer aus der benachbarten Elfenbeinküste waren als Austauschkontingent für eine andere Einheit nach Mali geflogen, wie die Nachrichtenagentur dpa unter Berufung auf Militärkreise berichtete. Nach Angaben von MINUSMA-Sprecher Olivier Salgado sollen die ivorischen Soldaten die Blauhelmmission unterstützen. Sie zählten zwar nicht zum Kontingent der Blauhelme, seien aber zur logistischen Unterstützung der UN-Truppen im Land. Das sei "gängige Praxis" bei Friedensmissionen. Die Soldaten sind Austausch für ei

100it [02:50,  1.71s/it]


100 Artikel in 2:36 Minuten - 1,56 Sekunden pro Artikel. Das ist nicht schnell genug. Wir brauchen mehr Parallelität!

In [42]:
# cluster = LocalCluster()
# client = Client(cluster)
# client

c:\Users\p\miniconda3\envs\mbb-asud-proto-tagesschau\Lib\site-packages\distributed\node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 56588 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:56588/status,
Dashboard: http://127.0.0.1:56588/status,Workers: 4
Total threads: 12,Total memory: 15.90 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:56590,Workers: 4
Dashboard: http://127.0.0.1:56588/status,Total threads: 12
Started: Just now,Total memory: 15.90 GiB
Comm: tcp://127.0.0.1:56625,Total threads: 3
Dashboard: http://127.0.0.1:56630/status,Memory: 3.98 GiB
Nanny: tcp://127.0.0.1:56593,


In [13]:
def do_extract_entities(df, partition_info: dict):
    file_name = f"{_dd}/2_5_processed-{partition_info['number']}.jsonl"

    with jl.open(file_name, "w") as f_out:
        for text in df.loc[:, ["url", "fulltext"]].itertuples():
            doc = nlp(text.fulltext)
            entities = entity_list(doc)
            for entity in entities:
                f_out.write(dict(url=text.url, **entity))

ddf_text = dd.from_pandas(df_texts, chunksize=200)

with ProgressBar():
    ddf_text.map_partitions(do_extract_entities, meta=("entities", "object")).compute()

[########################################] | 100% Completed | 33m 10s


Outputfiles zusammenführen:

In [14]:
with open(f"{_dd}/2_5_processed.jsonl", "wb") as f_out:
    for chunk in Path(_dd).glob("2_5_processed-*.jsonl"):
        with open(chunk,'rb') as f_in:
            shutil.copyfileobj(f_in, f_out)
        chunk.unlink()

JSONL -> Parquet:

In [21]:
df_found_entities = pd.read_json(f"{_dd}/2_5_processed.jsonl", lines=True)
df_found_entities.to_parquet(f"{_dd}/2_5_processed.parquet", index=False)
df_found_entities.sample(5)
# del(df_found_entities)

,url,ner_text,label,wd_id,wd_url,score
462202,/ausland/weltbevoelkerung-133.html,Afrika,LOC,Q15,https://www.wikidata.org/wiki/Q15,0.6299
453713,/inland/merkel-regierungserklaerung-corona-103...,Merkel,PER,Q567,https://www.wikidata.org/wiki/Q567,0.3106
532794,/ausland/interview-tuerkei-gipfel-101.html,Türkei,LOC,Q43,https://www.wikidata.org/wiki/Q43,0.6760
431195,/ausland/usa-einwanderung-107.html,Mexiko,LOC,Q96,https://www.wikidata.org/wiki/Q96,0.6736
943235,/newsticker/liveblog-ukraine-samstag-205.html,ukrainischer,MISC,Q212,https://www.wikidata.org/wiki/Q212,0.7078


### Entities von Wikidata scrapen

2. Entities in neues Dict, dabei Infos aus der NER beibehalten und Anreicherung durch Wikidata vorbereiten
3. Liste aller Entities für Scraper generieren (in jsonl oder so)
4. Alle Entities von Wikidata scrapen
   1. Herausforderung I: Manche Einträge haben kein Label/keine Description -> dann statt dessen Entitätskenner
   2. Herausforderung II: Manche Einträge leiten auf andere Entitäten weiter -> dann weitergeleitete ID speichern
5. Zusammenführen (direkt im Scraper?)

Der Datensatz enthält disambiguierte und niedrig bewertete Entitäten:

In [38]:
print(df_found_entities["wd_id"].notna().sum())
print(df_found_entities.shape)
print(df_found_entities.query("score < 0.1").shape)

996274
(996274, 6)
(118857, 6)


Die entfernen wir:

In [40]:
df_found_entities.dropna(inplace=True)
df_found_entities = df_found_entities[df_found_entities["score"] >= 0.1]

Der Rest darf bleiben:

In [41]:
print(df_found_entities.shape)

(877417, 6)


Entitätsliste für den Wikidata-Scraper generieren:

In [31]:
with open(f"{_dd}/3_catalog.json", "w") as f_out:
    json.dump(df_found_entities["wd_id"].sort_values().drop_duplicates().to_list(), f_out)

### Step X: Entitäten aus Wikidata scrapen

```bash
cd wdscrape
scrapy crawl wdspider -O 4_wd-catalog.jsonl
```

### Step X: Knoten und Kanten für Graphen generieren

Knoten sind die Entitäten aus `4_wd-catalog.jsonl`. Als technische Knoten-ID dient der numerische Teil der Wikidata-ID.

In [4]:
df_nodes_entities = pd.read_json(f"{_dd}/4_wd-catalog.jsonl", lines=True)
df_nodes_entities["node_id"] = df_nodes_entities["wd_entity_id"].str.lstrip("Q").astype(int)
df_nodes_entities.set_index("node_id", inplace=True)
df_nodes_entities.sample(5)

,wd_url,wd_entity_id,wd_label,wd_description,wd_redirect
node_id,,,,,
1584027,https://www.wikidata.org/wiki/Special:EntityDa...,Q1584027,Monika Krüger,German veterinarian,NaN
3741108,https://www.wikidata.org/wiki/Special:EntityDa...,Q3741108,Rimi Baltic,"company based in Riga, Latvia",NaN
7916262,https://www.wikidata.org/wiki/Special:EntityDa...,Q7916262,Varyl Begg,Royal Navy admiral of the fleet (1908-1995),NaN
189692,https://www.wikidata.org/wiki/Special:EntityDa...,Q189692,Genk,"municipality in Limburg, Belgium",NaN
15851383,https://www.wikidata.org/wiki/Special:EntityDa...,Q15851383,EU Transparency Register,Q15851383,NaN


Wir errechnen die Kanten aus `2_processed.parquet`. Wenn zwei Entitäten in einem Artikel vorkommen, wird eine Kante zwischen ihnen gezogen. Die Kanten werden gewichtet, indem die Anzahl der gemeinsamen Artikel als Gewicht verwendet wird. Dabei sind zwei Aspekte zu beachten:

- Wenn die Liste sowohl das Knotenpaar (159, 212) als auch (212, 159) enthalten kann, ist sie implizit gerichtet. Das wollen wir vermeiden, also sortieren wir die Knotenliste pro Artikel, bevor wir Knotenpaare bilden, so dass $ u \le v $.
- Die Liste wird viele parallele Kanten zwischen denselben Knoten enthalten, die in einem späteren Schritt gegebenenfalls aggregiert werden müssen

In [5]:
from itertools import combinations

df_mentions = pd.read_parquet(f"{_dd}/2_processed.parquet")
df_mentions.dropna(inplace=True)
df_mentions = df_mentions[df_mentions["score"] >= 0.1]
df_mentions["node_id"] = df_mentions["wd_id"].str.lstrip("Q").astype(int)

df_edges = pd.DataFrame(columns=["node1", "node2", "weight"])

url_mentions = []
for url, df in tqdm(df_mentions.groupby("url")):
    url_mentions.extend([
        {"u": node1, "v": node2, "weight": 1}
        for node1, node2 in combinations(np.sort(df["node_id"].unique().astype(int)), 2)
    ])

df_edges_entities = pd.DataFrame(url_mentions)

100%|██████████| 24918/24918 [00:05<00:00, 4813.63it/s]


Zur Behandlung des zweiten Aspekts aggregieren wir parallele Kanten und ermitteln ein Gewicht, das die Anzahl der gemeinsamen Artikel berücksichtigt:

In [6]:
df_aggregated_edges_entities = df_edges_entities.groupby(["u", "v"]).sum().reset_index()

In [8]:
print(f"{df_nodes_entities.shape=} {df_edges_entities.shape=} {df_aggregated_edges_entities.shape=}")

df_nodes_entities.shape=(50340, 5) df_edges_entities.shape=(7717210, 3) df_aggregated_edges_entities.shape=(3305861, 3)


In [8]:
df_nodes_entities.sample(5)

,wd_url,wd_entity_id,wd_label,wd_description,wd_redirect
node_id,,,,,
649416,https://www.wikidata.org/wiki/Special:EntityDa...,Q649416,bomb disposal,activity to dispose of and render safe explosi...,NaN
20850888,https://www.wikidata.org/wiki/Special:EntityDa...,Q20850888,Q20850888,Q20850888,NaN
555622,https://www.wikidata.org/wiki/Special:EntityDa...,Q555622,Eschede,municipality of Germany,NaN
87194236,https://www.wikidata.org/wiki/Special:EntityDa...,Q87194236,Susanne Johna,German physician,NaN
108756275,https://www.wikidata.org/wiki/Special:EntityDa...,Q108756275,Philipp Hartewig,german politician,NaN


In [9]:
df_aggregated_edges_entities.sort_values(by="weight", ascending=False).head(5)

,u,v,weight
158836,46,183,3536
319344,159,212,3164
343138,183,458,3150
158946,46,458,2819
319333,159,183,2664


## Graph aufbauen

### Mit @sw-NetworkX

#### Aggregierte Kanten

In [70]:
nxga = nx.Graph()

for row in tqdm(df_nodes_entities.reset_index().itertuples()):
    nxga.add_node(row.node_id, name=row.wd_label, label=row.wd_label, description=row.wd_description, url=row.wd_url, entity_id=row.wd_entity_id)

for row in tqdm(df_aggregated_edges_entities.itertuples()):
    nxga.add_edge(row.u, row.v, weight=row.weight)

nx.write_gml(nxga, f"{_dd}/5_graph_aggregated.gml")

50340it [00:00, 338293.14it/s]
3305861it [00:19, 167723.95it/s]


#### Nicht aggregierte Kantern

In [7]:
# del(nxgu)
nxgu = nx.Graph()

for row in tqdm(df_nodes_entities.reset_index().itertuples()):
    nxgu.add_node(row.node_id, name=row.wd_label, label=row.wd_label, description=row.wd_description, url=row.wd_url, entity_id=row.wd_entity_id)

for row in tqdm(df_edges_entities.itertuples()):
    nxgu.add_edge(row.u, row.v, weight=row.weight)

# nx.write_gml(nxgu, f"{_dd}/5_graph_not_aggregated.gml")

50340it [00:00, 271375.85it/s]
7717210it [00:23, 334364.77it/s]


### Visualisierung

In [29]:
import altair as alt
import matplotlib.pyplot as plt

plt.hist(nx.degree_histogram(nxgu))
plt.show()

# degree_sequence = sorted((d for n, d in nxgu.degree()), reverse=True)
# degree_histogram = nx.degree_histogram(nxgu)
# degree_df = pd.DataFrame(degree_histogram, columns=["count"])
# degree_df = pd.cut(degree_df.index, bins=range(1, max(degree_histogram)-5, 5), right=False)

# alt.Chart(degree_df).mark_bar().encode(
#     alt.X("IMDB_Rating:Q", bin=True),
#     y='count()',
# )


: 

: 

In [3]:
nxga = nx.read_gml(f"{_dd}/5_graph_aggregated.gml")

KeyboardInterrupt: 

In [11]:
nx.eigenvector_centrality(nxg)

{'Q43': 0.05218088650833873,
 'Q228389': 0.0073344977183745705,
 'Q2124692': 0.0009897912688900119,
 'Q153770': 0.013742753771129093,
 'Q3640': 0.03252746762468419,
 'Q918': 0.0554062578167746,
 'Q1385730': 0.00023803062691614847,
 'Q261543': 0.01204187494027784,
 'Q677466': 0.03101693804634078,
 'Q1140115': 0.03035563422876894,
 'Q256': 0.018530871471240627,
 'Q1432594': 0.0001488813332540527,
 'Q29': 0.045004948845915986,
 'Q7781': 0.008188654209473461,
 'Q5831': 0.001687978383523578,
 'Q5756': 0.006526674446504163,
 'Q81949': 0.0003093599303131326,
 'Q15': 0.03947359635418088,
 'Q1028': 0.020696404391401982,
 'Q311440': 0.03706560236266525,
 'Q51977': 0.03127697195769522,
 'Q8798': 0.04695560013239372,
 'Q515': 0.051061236279913574,
 'Q159': 0.06374764760833994,
 'Q33280': 0.021729231777012515,
 'Q44806': 0.05166489352486208,
 'Q134279': 0.0397910376394532,
 'Q3874799': 0.05415948606746713,
 'Q1221156': 0.05931344626104993,
 'Q936105': 0.05043660153010896,
 'Q42308': 0.0436375382973

In [12]:
nx.voterank(ng)

KeyboardInterrupt: 

### Mit neo4j

Wir verwenden CSV-Dateien, um den Graph in neo4j zu importieren.

In [11]:
print(f"{df_nodes_entities.shape=} {df_edges_entities.shape=} {df_aggregated_edges_entities.shape=}")

df_nodes_entities.to_csv(f"{_dd}/5_neo4j_nodes.csv", index=True, header=True)
df_aggregated_edges_entities.to_csv(f"{_dd}/5_neo4j_aggregated_edges.csv", index=False, header=True)
df_edges_entities.to_csv(f"{_dd}/5_neo4j_not_aggregated_edges.csv", index=False, header=True)

df_nodes_entities.shape=(50340, 5) df_edges_entities.shape=(7717210, 3) df_aggregated_edges_entities.shape=(3305861, 3)


Import der Knoten in neo4j:

```cypher
LOAD CSV WITH HEADERS FROM 'file:///5_neo4j_nodes.csv' AS row 
WITH row
MERGE (e:Entity {id: row.node_id, entityId: row.wd_entity_id, label: row.wd_label, description: row.wd_description, url: row.wd_url});
```

Index anlegen:

```cypher
CREATE INDEX nr_entity_id_index FOR (e:Entity) ON (e.id);
```

Import der Kanten in neo4j (:auto und CALL {} IN TRANSACTIONS splitten den Import auf, um Speicherprobleme zu vermeiden):

```cypher
:auto LOAD CSV WITH HEADERS FROM 'file:///5_neo4j_edges.csv' AS row 
CALL {
    WITH row
    MATCH (u:Entity {id: row.u})
    MATCH (v:Entity {id: row.v})
    MERGE (u)-[:COMENTION {count: toInteger(row.weight)}]-(v)
} IN TRANSACTIONS;
```

Alle Kanten löschen:

```cypher
:auto MATCH (u:Entity)-[c:COMENTION]-(v:Entity) CALL { WITH c DELETE c } IN TRANSACTIONS;
```

Knoten mit vielen Verbindungen anzeigen:

```cypher
MATCH (u)-[c:COMENTION]-(v) where c.count > 1000 RETURN u,v;
```

## Mit `graph-tool` (erfordert Linux/MacOS)

### Aggregierte Kanten mit Gewicht

In [71]:
gg = gt.load_graph(f"{_dd}/5_graph_aggregated.gml")

In [ ]:
ev, gv = gt.eigenvector(gg)

In [14]:
print(f"{ev=}, {gv=}")

ev=1213.7847366729438, gv=<VertexPropertyMap object with value type 'double', for Graph 0x7eff051606d0, at 0x7eff0518a5d0>


In [40]:
gg.ep

{'weight': <EdgePropertyMap object with value type 'double', for Graph 0x7efee2a41510, at 0x7efef3180850>}

In [72]:
deg = gg.degree_property_map("total", weight=gg.ep.weight)
print(deg.get_array())

[   5. 4018.    9. ...  307.  533.    6.]


In [73]:
gg_high_degree = gt.GraphView(gg, vfilt=deg.get_array() > 70000)
gg_high_degree
gg.vp

{'description': <VertexPropertyMap object with value type 'string', for Graph 0x7efc3a26b310, at 0x7efc39354d10>, 'entity\x00id': <VertexPropertyMap object with value type 'string', for Graph 0x7efc3a26b310, at 0x7efc797347d0>, 'label': <VertexPropertyMap object with value type 'string', for Graph 0x7efc3a26b310, at 0x7efc57785690>, 'name': <VertexPropertyMap object with value type 'string', for Graph 0x7efc3a26b310, at 0x7efc49dd0d90>, 'url': <VertexPropertyMap object with value type 'string', for Graph 0x7efc3a26b310, at 0x7efc5d8d73d0>}

In [135]:
hd_deg = gg_high_degree.degree_property_map("total", weight=gg_high_degree.ep.weight)
pos = gt.radial_tree_layout(gg_high_degree, gg_high_degree.vertices().next())
# pos = gt.sfdp_layout(gg_high_degree)
gt.graph_draw(
    gg_high_degree,
    pos=pos,
    output_size=(600, 1200),
    # output_size=(1200, 600),
    vertex_text=gg.vp.name,
    vertex_text_position=-2,
    vertex_fill_color="#99DDFF",
    vertex_font_size=16,
    vertex_size=100,
    vertex_pen_width=2,
    edge_pen_width=2,
    edge_color="#DDDDDD",
    output="newsgraph.png"
    # vertex_halo_size=10
)

<VertexPropertyMap object with value type 'vector<double>', for Graph 0x7efc3b00b810, at 0x7efc312ea990>

### Unaggregierte Kanten ohne Gewicht

In [19]:
gng = gt.load_graph(f"{_dd}/5_graph_not_aggregated.gml")
gng

<Graph object, undirected, with 50342 vertices and 7717210 edges, 4 internal vertex properties, 2 internal edge properties, 1 internal graph property, at 0x7eff05157690>

In [26]:
gng_high_degree = gt.GraphView(gng, vfilt=lambda v: v.in_degree() + v.out_degree() > 1000)
gng_high_degree

<GraphView object, undirected, with 1984 vertices and 4413679 edges, 4 internal vertex properties, 2 internal edge properties, 1 internal graph property, edges filtered by (<EdgePropertyMap object with value type 'bool', for Graph 0x7efee2b9f910, at 0x7efee2bf4b50>, False), vertices filtered by (<VertexPropertyMap object with value type 'bool', for Graph 0x7efee2b9f910, at 0x7efee2a61b90>, False), at 0x7efee2b9f910>

In [34]:
deg = gng.degree_property_map("total").get_array()
gng_high_degree2 = gt.GraphView(gng, vfilt=deg > 40000)
gng_high_degree2

<GraphView object, undirected, with 40 vertices and 318739 edges, 4 internal vertex properties, 2 internal edge properties, 1 internal graph property, edges filtered by (<EdgePropertyMap object with value type 'bool', for Graph 0x7efcea11fa10, at 0x7efee2e50fd0>, False), vertices filtered by (<VertexPropertyMap object with value type 'bool', for Graph 0x7efcea11fa10, at 0x7efee2e505d0>, False), at 0x7efcea11fa10>